In [3]:
import pandas as pd
import sys
sys.path.append('/home/matta/recipy_maker')
import consume       

# 


In [2]:
my_data = pd.read_pickle('../Web_Scraping/df_w_json.pkl')

my_data.head(1)

,title,image_url,description,ingredients,instructions,json
0,Traditional Mexican Guacamole,https://imagesvc.meredithcorp.io/v3/mm/image?u...,This guacamole is great! Whether or not you wa...,"[2 avocados, peeled and pitted , 1 cup chopped...",Mash avocados in a bowl until creamy.\nMix tom...,"{'title': 'Traditional Mexican Guacamole', 'im..."


In [3]:
my_input = my_data[150:]
my_input.head(1)
# maybe we could/ should add a tag --- would be in scraping part.. whatever category it came from

,title,image_url,description,ingredients,instructions,json
150,Basic Air Fryer Hot Dogs,https://imagesvc.meredithcorp.io/v3/mm/image?u...,Perfectly crisp hot dogs and toasted buns made...,"[4 hot dog buns , 4 hot dogs ]",Preheat air fryer to 390 degrees F (200 degree...,"{'title': 'Basic Air Fryer Hot Dogs', 'image_u..."


In [4]:
# for index,row in my_input.iterrows():
#     title = row['title']
#     image_url = row['image_url']
#     description = row['description']
#     ingredient_list = row['ingredients']
#     instructions = row['instructions']
#     r_api.create_recipe(title,description,ingredient_list,instructions,image_url)

In [4]:
r_api = consume.API('admin','1477')


In [6]:
# import names
# import random

# name_list = []

# for _ in range(20):
#     name = names.get_first_name()
#     number = str(random.randint(0,100))
#     username = name + number
#     name_list.append(username)

In [17]:
# user_list = []
# for name in name_list:
#     api_access = consume.API()
#     email = name + "@email.com"
#     email2 = name + "@email.com"
#     password = "Uns4fepassword12"
#     #api_access.create_user(name,email,email2,password)
#     user_list.append({"name":name,"password":password})

In [23]:
# import pickle

# with open('user_list.pkl','wb') as f:
#     pickle.dump(user_list,f)

In [6]:
import pickle
with open('user_list.pkl','rb') as f:
    user_list = pickle.load(f)



In [7]:
for user in user_list:
    username = user['name']
    password = user['password']
    api_access = consume.API(username,password)
    

In [9]:
r_api = consume.API()
recipe_list = []
for i in range(31):
    m_list = r_api.get_recipes(i+1)
    for i in range(len(m_list)):
        a = m_list[i].get('id')
        b = m_list[i].get('favorites')
        recipe_list.append([a,b])

In [10]:
my_recipe_df = pd.DataFrame(recipe_list,columns=['id','favorites']).explode('favorites')

username = user_list[0]['name']
print(username)
password = user_list[0]['password']
api_access = consume.API(username,password)

my_recipe_df.head(5)

Daniel85


,id,favorites
0,1,1
0,1,14
0,1,15
0,1,36
1,3,1


In [25]:
api_access.favorite_recipe(1)


'Recipe 1 ADDED to favorites'

In [21]:
for id_ in my_recipe_df['id']:
    api_access.favorite_recipe(id_)


In [25]:
df_recipe_crosstab = pd.crosstab(index=my_recipe_df['id'],columns=my_recipe_df['favorites'])

In [35]:
df_recipe_crosstab

favorites,1,3,13,14,15,36
id,,,,,,
1,1,0,0,1,1,1
3,1,0,1,0,0,1
4,1,1,1,0,1,1
5,0,0,1,1,0,1
6,1,0,1,0,1,1
...,...,...,...,...,...,...
617,0,0,0,0,0,1
618,0,0,0,0,0,1
619,0,0,0,0,0,1


In [27]:
matrix = df_recipe_crosstab.to_numpy()

In [28]:
matrix

array([[1, 0, 0, 1, 1, 1],
       [1, 0, 1, 0, 0, 1],
       [1, 1, 1, 0, 1, 1],
       ...,
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1]])

In [29]:
from sklearn.neighbors import NearestNeighbors
knn_model = NearestNeighbors(metric='cosine',algorithm='brute')
fit_model = knn_model.fit(matrix)

In [49]:
def get_user_recipes(user_id):
    return df_recipe_crosstab.index[df_recipe_crosstab[user_id]==True].tolist()

get_user_recipes(1)

[1, 3, 4, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 119]

In [ ]:
for recipe_id in get_user_recipes(1):
    

In [54]:
df_recipe_crosstab.loc[1].values.reshape(1,-1)

array([[1, 0, 0, 1, 1, 1]])

In [37]:
distances,indices = fit_model.kneighbors(input_,n_neighbors=5)

In [ ]:
# 1
# Compare most similar users to a selected user
# then return the most frequent recipes liked by other users which arent already favorite by selected user

# 2
# Compare most simliarly liked recipes for all recipes given a selected users
# Return most similar recipes which arent already favorited by a selected user